<a href="https://colab.research.google.com/github/luciana11levy/coder/blob/main/Data_Science_III_Poemas_Luciana_Levy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! python -m spacy download es_core_news_sm
! pip install -U symspellpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 57.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.1/144.1 kB 8.0 MB/s eta 0:00:00


In [2]:
import nltk # importar natural language toolkit
nltk.download('punkt')
nltk.download('stopwords') # modulo para descargar stopwords en diferentes idiomas
nltk.download('wordnet')
from nltk.corpus import stopwords
import pandas as pd
import numpy  as np
import re
import string
import plotly
import matplotlib.pyplot as plt
from nltk.stem import PorterStemmer
import time
import spacy
import es_core_news_sm
from nltk.tokenize import sent_tokenize, word_tokenize

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from nltk.probability import FreqDist
from wordcloud import WordCloud
import pickle
from symspellpy import SymSpell
import pkg_resources
from symspellpy import SymSpell, Verbosity

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
<ipython-input-2-cc7f8c20a4bf>:23: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources


In [12]:
from google.colab import drive
drive.mount('/content/drive')
import zipfile

with zipfile.ZipFile('/content/sample_data/poetry/PoetryFoundationData.csv.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/sample_data/poetry')  # Reemplaza '/content/destino' con la ruta donde deseas descomprimir los archivos

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:

import pandas as pd

# Lee el archivo CSV
df = pd.read_csv('/content/sample_data/poetry/PoetryFoundationData.csv')

#Mostrar las primeras filas del contenido
print(df.head())

   Unnamed: 0                                              Title  \
0           0  \r\r\n                    Objects Used to Prop...   
1           1  \r\r\n                    The New Church\r\r\n...   
2           2  \r\r\n                    Look for Me\r\r\n   ...   
3           3  \r\r\n                    Wild Life\r\r\n     ...   
4           4  \r\r\n                    Umbrella\r\r\n      ...   

                                                Poem              Poet Tags  
0  \r\r\nDog bone, stapler,\r\r\ncribbage board, ...  Michelle Menting  NaN  
1  \r\r\nThe old cupola glinted above the clouds,...     Lucia Cherciu  NaN  
2  \r\r\nLook for me under the hood\r\r\nof that ...        Ted Kooser  NaN  
3  \r\r\nBehind the silo, the Mother Rabbit\r\r\n...   Grace Cavalieri  NaN  
4  \r\r\nWhen I push your button\r\r\nyou fly off...      Connie Wanek  NaN  


In [31]:
 #1. Contar la cantidad de poemas por autor
print("\nCantidad de poemas por autor:")
print(df['Poet'].value_counts())



Cantidad de poemas por autor:
Poet
William Shakespeare      85
Alfred, Lord Tennyson    73
Emily Dickinson          51
William Wordsworth       51
Rae Armantrout           49
                         ..
George Kalogeris          1
Gowann                    1
Seán Hewitt               1
Michelle Boisseau         1
Nadra Mabrouk             1
Name: count, Length: 3128, dtype: int64


In [32]:
# 2. Mostrar los 10 poemas más largos (por cantidad de palabras)
df['word_count'] = df['Poem'].apply(lambda x: len(x.split()))
print("\nLos 10 poemas más largos:")
print(df.sort_values('word_count', ascending=False).head(10)[['Title', 'Poet', 'word_count']])



Los 10 poemas más largos:
                                                   Title                 Poet  \
13065  \r\r\n                    Song of Myself (1892...         Walt Whitman   
2392   \r\r\n                    Venus and Adonis\r\r...  William Shakespeare   
9330   \r\r\n                    Scenes of Life at th...        Philip Whalen   
7359   \r\r\nParadise Lost: Book  9 (1674 version)\r\...          John Milton   
897    \r\r\nParadise Lost: Book 10 (1674 version)\r\...          John Milton   
891    \r\r\nParadise Lost: Book  2 (1674 version)\r\...          John Milton   
8411   \r\r\n                    The Glass Essay\r\r\...          Anne Carson   
893    \r\r\nParadise Lost: Book  4 (1674 version)\r\...          John Milton   
7033   \r\r\n                    Absalom and Achitoph...          John Dryden   
12554  \r\r\n                    Sohrab and Rustum\r\...       Matthew Arnold   

       word_count  
13065       15713  
2392         9713  
9330         9451  
7

In [33]:
# Análisis de frecuencia de palabras usando NLTK
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
nltk.download('punkt_tab')

all_words = []
for poem in df['Poem']:
    words = word_tokenize(poem.lower())
    words = [w for w in words if w.isalnum() and w not in stop_words]
    all_words.extend(words)

word_freq = nltk.FreqDist(all_words)

print("\nPalabras más frecuentes:")
print(word_freq.most_common(20)) # Muestra las 20 palabras más comunes


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!



Palabras más frecuentes:
[('like', 14819), ('one', 11787), ('us', 6154), ('love', 5785), ('would', 5552), ('time', 5010), ('man', 4883), ('know', 4867), ('back', 4864), ('could', 4835), ('see', 4804), ('night', 4653), ('day', 4522), ('still', 4472), ('light', 4389), ('never', 4191), ('life', 4166), ('said', 4160), ('eyes', 4059), ('way', 3943)]


In [29]:
# Lemmatización

import nltk
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

df['lemmatized_poem'] = df['Poem'].apply(lambda poem: ' '.join([lemmatizer.lemmatize(word) for word in word_tokenize(poem.lower())]))

print(df[['Poem', 'lemmatized_poem']].head())


[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


                                                Poem  \
0  \r\r\nDog bone, stapler,\r\r\ncribbage board, ...   
1  \r\r\nThe old cupola glinted above the clouds,...   
2  \r\r\nLook for me under the hood\r\r\nof that ...   
3  \r\r\nBehind the silo, the Mother Rabbit\r\r\n...   
4  \r\r\nWhen I push your button\r\r\nyou fly off...   

                                     lemmatized_poem  
0  dog bone , stapler , cribbage board , garlic p...  
1  the old cupola glinted above the cloud , shone...  
2  look for me under the hood of that old chevrol...  
3  behind the silo , the mother rabbit hunch like...  
4  when i push your button you fly off the handle...  
